In [4]:
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv() 

openai.api_key = os.getenv("OPENAI_API_KEY")


client = OpenAI()

In [5]:

# respponse = client.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {"role": "user", "content": "What is human life expectancy in the United States?"}
#     ]   
# )   
# print(respponse.choices[0].message.content)

In [6]:
import requests
from io import BytesIO


def create_file(client, file_path):
    with open(file_path, "rb") as file_content:
            result = client.files.create(
                file=file_content,
                purpose="assistants"
            )
    # print(result.id)
    return result.id

file_id = create_file(client, "user.txt")

vector_store = client.vector_stores.create(
    name="knowledge_base"
)
print(vector_store.id)

result = client.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file_id
)
result = client.vector_stores.files.list(
    vector_store_id=vector_store.id
)
print(result)

vs_67d7e4386bb48191b3a42e5fd3613cde
SyncCursorPage[VectorStoreFile](data=[VectorStoreFile(id='file-SEZ3jf2EYcKyYyzr9XRD3q', created_at=1742201914, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_67d7e4386bb48191b3a42e5fd3613cde', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))], has_more=False, object='list', first_id='file-SEZ3jf2EYcKyYyzr9XRD3q', last_id='file-SEZ3jf2EYcKyYyzr9XRD3q')


In [ ]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="what is the name and rating of the user?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": ["vs_67d747b5c0a48191852b145388d0049d"]
    }]z
)
print(response.output[1].content[0].text)

The user's name is **Alex Smith** and their ratings in competitive programming are as follows:

- **Codeforces:** 1020
- **Codechef:** 980
- **AtCoder:** 850
- **LeetCode:** Success Rate: 65% with a 15-day streak.


In [11]:
def get_response(query):
    response = client.responses.create( 
        model="gpt-4o-mini",
        input=query,
        instructions="""
        You're a Competitive Programming Mentor. You're helping a student with a problem. 
        The student is stuck on a problem and asks for help.
        When asked for problem suggestions, suggest problems from these sites:
        Codeforces, Codechef, Atcoder, Leetcode, Hackerrank, Topcoder, and A2OJ Ladder.
        Consider the user's rating, problem difficulty (easy, medium, hard), and problem tags 
        (such as dp, greedy, graph, math, etc.) when suggesting problems.
        - Use the file_search tool to search for the user's file.
        - Use the web_search_preview tool to search for problems. include citations.
        """,
        tools=[
            {
                "type": "file_search",
                "vector_store_ids": ["vs_67d747b5c0a48191852b145388d0049d",],
                # "filters": {
                #     "type": "eq",
                #     "key": "username",
                #     "value": "Alex smith"
                # }
            },
            {
                "type": "web_search_preview"
            }
        ],
        # stream=True
    )
    return response

# Get the streaming response object
stream = get_response("suggest me three problems")

# Iterate through the stream to print out the results
# print(stream.output[1].content[0].text)
import rich
rich.print(stream)
print(stream)

Response(
    id='resp_67d7e4ab8a648191980f5567d10d7bab08c03e0bd544c71b',
    created_at=1742202027.0,
    error=None,
    incomplete_details=None,
    instructions="\n        You're a Competitive Programming Mentor. You're helping a student with a problem. \n   
The student is stuck on a problem and asks for help.\n        When asked for problem suggestions, suggest problems 
from these sites:\n        Codeforces, Codechef, Atcoder, Leetcode, Hackerrank, Topcoder, and A2OJ Ladder.\n       
Consider the user's rating, problem difficulty (easy, medium, hard), and problem tags \n        (such as dp, 
greedy, graph, math, etc.) when suggesting problems.\n        - Use the file_search tool to search for the user's 
file.\n        - Use the web_search_preview tool to search for problems. include citations.\n        ",
    metadata={},
    model='gpt-4o-mini-2024-07-18',
    object='response',
    output=[
        ResponseFileSearchToolCall(
            id='fs_67d7e4abf940819180004eff9312f3e908c03e0bd544c71b',
            queries=['Suggest me three problems', 'problems', 'competitive programming problems'],
            status='completed',
            type='file_search_call',
            results=None
        ),
        ResponseFunctionWebSearch(
            id='ws_67d7e4ad4eec8191ac1ead9da097711508c03e0bd544c71b',
            status='completed',
            type='web_search_call'
        ),
        ResponseOutputMessage(
            id='msg_67d7e4af9b70819196ca4a17b966b79708c03e0bd544c71b',
            content=[
                ResponseOutputText(
                    annotations=[],
                    text='Based on your competitive programming profile, here are three problems that align with 
your strengths and areas for improvement:\n\n1. **Problem: "A. Square String?"**\n   - **Platform:** Codeforces\n  
- **Difficulty:** Easy\n   - **Tags:** Implementation, Strings\n   - **Description:** Determine if a given string 
is a "square string," meaning it can be formed by repeating a substring twice. This problem tests your 
understanding of string manipulation and basic implementation skills.\n\n2. **Problem: "Apple Division"**\n   - 
**Platform:** Codeforces\n   - **Difficulty:** Medium\n   - **Tags:** Dynamic Programming, Greedy Algorithms\n   - 
**Description:** Given a set of apples with known weights, divide them into two groups such that the difference in 
total weight between the groups is minimized. This problem involves dynamic programming and greedy algorithms, 
which are areas you\'re aiming to strengthen.\n\n3. **Problem: "Movie Festival"**\n   - **Platform:** Codeforces\n 
- **Difficulty:** Medium\n   - **Tags:** Greedy Algorithms, Sorting\n   - **Description:** In a movie festival, you
have a list of movies with their start and end times. Determine the maximum number of movies you can watch entirely
without overlap. This problem requires efficient sorting and greedy selection strategies, helping you improve your 
optimization skills.\n\nThese problems are selected to build upon your existing strengths and address areas where 
you\'re seeking improvement. Tackling them will enhance your problem-solving abilities and prepare you for more 
advanced challenges. ',
                    type='output_text'
                )
            ],
            role='assistant',
            status='completed',
            type='message'
        )
    ],
    parallel_tool_calls=True,
    temperature=1.0,
    tool_choice='auto',
    tools=[
        FileSearchTool(
            type='file_search',
            vector_store_ids=['vs_67d747b5c0a48191852b145388d0049d'],
            filters=None,
            max_num_results=20,
            ranking_options=RankingOptions(ranker='auto', score_threshold=0.0)
        ),
        WebSearchTool(
            type='web_search_preview',
            search_context_size='medium',
            user_location=UserLocation(type='approximate', city=None, country='US', region=None, timezone=None)
        )
    ],
   

Response(id='resp_67d7e4ab8a648191980f5567d10d7bab08c03e0bd544c71b', created_at=1742202027.0, error=None, incomplete_details=None, instructions="\n        You're a Competitive Programming Mentor. You're helping a student with a problem. \n        The student is stuck on a problem and asks for help.\n        When asked for problem suggestions, suggest problems from these sites:\n        Codeforces, Codechef, Atcoder, Leetcode, Hackerrank, Topcoder, and A2OJ Ladder.\n        Consider the user's rating, problem difficulty (easy, medium, hard), and problem tags \n        (such as dp, greedy, graph, math, etc.) when suggesting problems.\n        - Use the file_search tool to search for the user's file.\n        - Use the web_search_preview tool to search for problems. include citations.\n        ", metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseFileSearchToolCall(id='fs_67d7e4abf940819180004eff9312f3e908c03e0bd544c71b', queries=['Suggest me three problems', 

In [ ]:
from playwright.async_api import async_playwright


with async_playwright() as p:
    browser = p.chromium.launch(
        headless=False,
        chromium_sandbox=True,
        env={},
        args=[
            "--disable-extensions",
            "--disable-file-system"
        ]
    )
    page = browser.new_page()
    page.set_viewport_size({"width": 1024, "height": 768})
    page.goto("https://codeforces.com")

    page.wait_for_timeout(10000)

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [ ]:
def get_response(query):
    response = client.responses.create( 
        model="computer-use-preview",
        input=query,
        instructions="""
        You're a Competitive Programming Mentor. You're helping a student with a problem. 
        The student is stuck on a problem and asks for help.
        When asked for problem suggestions, suggest problems from these sites:
        Codeforces, Codechef, Atcoder, Leetcode, Hackerrank, Topcoder, and A2OJ Ladder.
        Consider the user's rating, problem difficulty (easy, medium, hard), and problem tags 
        (such as dp, greedy, graph, math, etc.) when suggesting problems.
        - Use the file_search tool to search for the user's file.
        - Use the web_search_preview tool to search for problems. include citations.
        """,
        truncation="auto",
        tools=[
            {
                "type": "computer_use_preview",
                "display_height": 1024,
                "display_width": 768,
                "environment" : "browser"
                
            }
        ],
        
    input=[
        {
            "role": "user",
            "content": query
        }
        # Optional: include a screenshot of the initial state of the environment
        # {
        #     type: "input_image",
        #     image_url: f"data:image/png;base64,{screenshot_base64}"
        # }
    ],
    reasoning={
        "generate_summary": "concise",
    },
        # stream=True
    )
    return response

# Get the streaming response object
stream = get_response("suggest me three problems")

# Iterate through the stream to print out the results
# print(stream.output[1].content[0].text)
print(stream)

Response(id='resp_67d75be629408191be5336b51bcdeab704226677997bdc31', created_at=1742167014.0, error=None, incomplete_details=None, instructions="\n        You're a Competitive Programming Mentor. You're helping a student with a problem. \n        The student is stuck on a problem and asks for help.\n        When asked for problem suggestions, suggest problems from these sites:\n        Codeforces, Codechef, Atcoder, Leetcode, Hackerrank, Topcoder, and A2OJ Ladder.\n        Consider the user's rating, problem difficulty (easy, medium, hard), and problem tags \n        (such as dp, greedy, graph, math, etc.) when suggesting problems.\n        - Use the file_search tool to search for the user's file.\n        - Use the web_search_preview tool to search for problems. include citations.\n        ", metadata={}, model='computer-use-preview-2025-03-11', object='response', output=[ResponseOutputMessage(id='msg_67d75be722dc81919096d1d4b79685a704226677997bdc31', content=[ResponseOutputText(annot